<a href="https://colab.research.google.com/github/raminass/deep-NMF/blob/master/comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/My Drive/deep_nmf

In [ ]:
import utils
import numpy as np
from sklearn.decomposition import NMF
import sklearn.decomposition as sk_deco
import torch.optim as optim
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import torch
from my_layers import *
from matplotlib import pyplot as plt

#  Execute all 'print' statments of a cell instead of only the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Data Loading
M = np.load('synthetic_data/x.syn.many.types.0.5_sp.sp.npy')
X = M.T

In [ ]:
#params
n_components = 21  # from summary table
samples, features = X.shape

In [ ]:
W_init, H_init = sk_deco._nmf._initialize_nmf(X, n_components, init='nndsvd')

In [ ]:
#split train/test
TRAIN_SIZE = 0.80
mask = np.random.rand(samples) < TRAIN_SIZE

X_train = X[mask]  
X_test = X[~mask] 

In [ ]:
# MU building target labels for training using Scikit NMF
nmf_mu = NMF(n_components=n_components, solver='mu', beta_loss='frobenius', init='custom')
W_train_mu = nmf_mu.fit_transform(X_train,W=W_init[mask],H=H_init)
H_mu = nmf_mu.components_
W_test_mu = nmf_mu.transform(X_test)

In [ ]:
nmf_cd = NMF(n_components=n_components, solver='cd', beta_loss='frobenius', init='custom')
W_train_cd = nmf_cd.fit_transform(X_train,W=W_init[mask],H=H_init)
H_cd = nmf_cd.components_
W_test_cd = nmf_cd.transform(X_test)

In [ ]:
W_bcd, H_bcd = utils.sBCD_update(X_train, W_init[mask], H_init, O=1,obj='euc')

In [ ]:
print(sk_deco._nmf._beta_divergence(X_train, W_train_cd, H_cd, 2,square_root=True))
print(sk_deco._nmf._beta_divergence(X_train, W_train_mu, H_mu, 2,square_root=True))
print(sk_deco._nmf._beta_divergence(X_train, W_bcd, H_bcd, 2,square_root=True))

In [ ]:
sk_deco._nmf._beta_divergence(X_test, W_test_cd, H_cd, 2,square_root=True)
sk_deco._nmf._beta_divergence(X_test, W_test_mu, H_mu, 2,square_root=True)

In [ ]:
# MU building target labels for training using Scikit NMF
nmf_mu = NMF(n_components=n_components, solver='mu', beta_loss='kullback-leibler', init='custom')
W_train_mu = nmf_mu.fit_transform(X_train,W=W_init[mask],H=H_init)
H_mu = nmf_mu.components_
W_test_mu = nmf_mu.transform(X_test)

In [ ]:
W_bcd, H_bcd = utils.sBCD_update(X_train, W_init[mask], H_init, O=1,obj='kl')

In [ ]:
# sk_deco._nmf._beta_divergence(X_train, W_train_cd, H_cd, 1,square_root=True)
sk_deco._nmf._beta_divergence(X_train, W_train_mu, H_mu, 1,square_root=True)
sk_deco._nmf._beta_divergence(X_train, W_bcd, H_bcd, 1,square_root=True)

#### initialize exposures



In [ ]:
W0_train = W_init[mask]
W0_test = W_init[~mask] # might be per sample or include the whole X ??

#### Tensoring the Arrays

In [ ]:
X_train_tensor = torch.from_numpy(X_train).float()
W_train_tensor = torch.from_numpy(W_bcd).float()
W0_train_tensor = torch.from_numpy(W0_train).float()

X_test_tensor = torch.from_numpy(X_test).float()
# W_test_tensor = torch.from_numpy(W_test_cd).float()
W0_test_tensor = torch.from_numpy(W0_test).float()

## Basic Model
9 layers with non-negative constrains on weights

Trained with Graident decent

### Training The Network

In [ ]:
%%capture
constraints = utils.WeightClipper()
fr_nmf = MultiBetaDNMFNet(12,1, n_components, features)
fr_nmf.apply(constraints)
criterion = nn.MSELoss()    
optimizerSGD = optim.SGD(fr_nmf.parameters(), lr=1e-4)
optimizerADAM = optim.Adam(fr_nmf.parameters(), lr=1e-4)

In [ ]:
inputs = (W0_train_tensor, X_train_tensor)
loss_values = []
for i in range(1000):

    out = fr_nmf(*inputs)
    loss = criterion(out, W_train_tensor)
    # print(i, loss.item())

    optimizerADAM.zero_grad()
    loss.backward()
    optimizerADAM.step()

    fr_nmf.apply(constraints) # keep wieghts positive
    loss_values.append(loss.item())

In [ ]:
plt.plot(loss_values)

### Compare with Test Data
comparison is on the reconstruction Error

In [ ]:
test_inputs = (W0_test_tensor, X_test_tensor)
netwrok_prediction = deep_nmf_5(*test_inputs)

In [ ]:
network_error = utils.frobinuis_reconstruct_error(X_test_tensor, netwrok_prediction, H)
print('deep NMF Error: ', network_error)

In [ ]:
mu_error = utils.frobinuis_reconstruct_error(X_test_tensor, W_test_tensor, H)
print('regular MU Error: ', mu_error)